# Data Collection and Processing

This notebook is going to cover the data collection from pybaseball, as well as cleaning the data and picking features. The data will be focused on Red Sox hitters in the 2024 season. Pitcher data will be collected based on pitchers that the Red Sox hitters faced throughout the season.

In [2]:
import pandas as pd

In [3]:
from pybaseball import batting_stats

all_qualified_2024 = batting_stats(2024, qual=100)

red_sox_qualified_batters = all_qualified_2024[all_qualified_2024["Team"] == "BOS"]

red_sox_qualified_batters.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
41,24617,2024,Jarren Duran,BOS,27,160,671,735,191,108,...,113.9,225,0.437,515,0.161,0.266,0.271,0.448,0.338,6.4
30,17350,2024,Rafael Devers,BOS,27,138,525,601,143,76,...,114.7,201,0.523,384,0.123,0.266,0.272,0.509,0.364,3.9
84,23772,2024,Wilyer Abreu,BOS,25,132,399,447,101,51,...,114.4,139,0.498,279,0.157,0.282,0.229,0.418,0.317,2.8
35,15711,2024,Tyler O'Neill,BOS,29,113,411,473,99,50,...,113.1,123,0.484,254,0.164,0.308,0.213,0.480,0.339,2.1
214,27531,2024,David Hamilton,BOS,26,98,294,317,73,47,...,108.8,69,0.322,214,0.187,0.290,0.231,0.346,0.281,1.9


In [4]:
from pybaseball import playerid_lookup

red_sox_names = red_sox_qualified_batters["Name"].unique()

player_ids = []
for name in red_sox_names:
    first, last = name.split(" ", 1)
    res = playerid_lookup(last, first)
    if not res.empty:
        player_ids.append(res.iloc[0])

player_ids_df = pd.DataFrame(player_ids)

Gathering player lookup table. This may take a moment.


In [5]:
player_ids_df

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,duran,jarren,680776,duraj001,duranja01,24617,2021.0,2025.0
0,devers,rafael,646240,dever001,deverra01,17350,2017.0,2025.0
0,abreu,wilyer,677800,abrew002,abreuwi02,23772,2023.0,2025.0
0,o'neill,tyler,641933,oneit001,oneilty01,15711,2018.0,2025.0
0,hamilton,david,666152,hamid002,hamilda03,27531,2023.0,2025.0
0,refsnyder,rob,608701,refsr001,refsnro01,13770,2015.0,2025.0
0,wong,connor,657136,wongc001,wongco01,19896,2021.0,2025.0
0,rafaela,ceddanne,678882,rafac001,rafaece01,24262,2023.0,2025.0
0,yoshida,masataka,807799,yoshm002,yoshima02,31837,2023.0,2024.0
0,casas,triston,671213,casat001,casastr01,22514,2022.0,2025.0


In [6]:
player_ids_df.to_csv("qualified_red_sox_id_table.csv", index=False)

In [7]:
from pybaseball import statcast
import pybaseball.cache

pybaseball.cache.enable()

data = statcast('2024-04-01', '2024-10-30', team='TEX')

data.head()



This is a large query, it may take a moment to complete


  0%|          | 0/213 [00:00<?, ?it/s]c:\Users\jtn80\Code\mlb-hit-prediction\.venv\lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  0%|          | 1/213 [00:03<13:39,  3.87s/it]c:\Users\jtn80\Code\mlb-hit-prediction\.venv\lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  1%|          | 2/213 [00:03<05:49,  1.66s/it]c:\Users\jtn80\Code\mlb-hit-prediction\.venv\lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
73,SL,2024-09-29,83.5,-2.8,5.86,"Robertson, David",694359,502085,strikeout,swinging_strike,...,1,3,2,1,<NA>,<NA>,3.69,-1.45,1.45,43.6
75,FC,2024-09-29,94.9,-2.81,5.79,"Robertson, David",694359,502085,NaN,foul,...,1,3,2,1,<NA>,<NA>,1.17,-0.08,0.08,40.7
78,FC,2024-09-29,94.3,-2.87,5.77,"Robertson, David",694359,502085,NaN,foul,...,1,3,2,1,<NA>,<NA>,1.39,-0.2,0.2,40.3
79,KC,2024-09-29,86.3,-2.71,5.92,"Robertson, David",694359,502085,NaN,ball,...,1,3,2,1,<NA>,<NA>,3.81,-0.62,0.62,44.5
81,FC,2024-09-29,93.9,-2.78,5.86,"Robertson, David",694359,502085,NaN,foul,...,1,3,2,1,<NA>,<NA>,1.42,-0.4,0.4,41.8


In [10]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description',
       ...
       'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
       'pitcher_days_since_prev_game', 'batter_days_since_prev_game',
       'pitcher_days_until_next_game', 'batter_days_until_next_game',
       'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in',
       'arm_angle'],
      dtype='object', length=113)

In [11]:
from pybaseball import batting_stats_range

df = batting_stats_range("2024-04-01", "2024-05-01")

df.head()

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
1,CJ Abrams,23,349,Maj-NL,Washington,24,109,99,18,29,...,0,0,0,4,2,0.293,0.358,0.586,0.944,682928
2,Jos\xc3\xa9 Abreu,37,352,Maj-AL,Houston,18,64,60,5,7,...,0,1,2,0,0,0.117,0.156,0.133,0.290,547989
3,Wilyer Abreu,25,349,Maj-AL,Boston,23,83,72,15,24,...,0,1,0,5,0,0.333,0.410,0.556,0.965,677800
4,Ronald Acu\xc3\xb1a Jr.,26,349,Maj-NL,Atlanta,26,122,103,25,25,...,0,0,2,12,1,0.243,0.361,0.320,0.681,660670
5,Willy Adames,28,349,Maj-NL,Milwaukee,27,120,103,16,28,...,0,1,1,4,1,0.272,0.367,0.505,0.872,642715


In [1]:
from pybaseball import pitching_stats

pitcher_2024 = pitching_stats(2024, qual=10)

pitcher_2024.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,...,botOvr FC,botStf FC,botCmd FC,botOvr FS,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
65,10603,2024,Chris Sale,ATL,35,18,3,6.4,2.38,29,29,0,0,0,0,177.2,702,141,48,47,9,39,0,8,4,1,225,191,143,92,8,925,1893,2818,126,5,4,0,11.40,1.98,...,NaN,NaN,NaN,NaN,NaN,NaN,60,48,64,-0.49,100.0,111.0,110.0,NaN,NaN,NaN,97.0,108.0,100.0,95.0,109.0,108.0,114.0,114.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,111,110,NaN,NaN,NaN
66,22267,2024,Tarik Skubal,DET,27,18,4,5.9,2.39,31,31,0,0,0,0,192.0,753,142,54,51,15,35,0,9,2,0,228,218,171,88,22,883,1985,2868,134,13,4,2,10.69,1.64,...,NaN,NaN,NaN,NaN,NaN,NaN,61,59,61,-0.55,127.0,98.0,124.0,NaN,NaN,NaN,104.0,104.0,107.0,111.0,100.0,108.0,108.0,96.0,111.0,98.0,102.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,112,100,112,NaN,NaN,NaN
78,10310,2024,Zack Wheeler,PHI,34,16,7,5.4,2.57,32,32,0,0,0,0,200.0,787,139,62,57,20,52,0,8,8,0,224,213,195,92,27,1079,2057,3136,118,17,3,0,10.08,2.34,...,45.0,61.0,47.0,52.0,46.0,58.0,64,59,60,-0.74,NaN,NaN,NaN,126.0,88.0,106.0,109.0,110.0,118.0,117.0,103.0,122.0,108.0,109.0,117.0,NaN,NaN,NaN,97.0,95.0,96.0,94.0,101.0,91.0,110,104,113,NaN,NaN,NaN
149,21846,2024,Cole Ragans,KCR,26,11,9,4.9,3.14,32,32,1,0,0,0,186.1,762,146,71,65,15,67,1,6,11,0,223,187,186,86,18,1138,1980,3118,86,10,7,2,10.77,3.24,...,51.0,55.0,56.0,NaN,NaN,NaN,58,62,54,-0.38,115.0,92.0,110.0,NaN,NaN,NaN,110.0,101.0,111.0,NaN,NaN,NaN,124.0,102.0,117.0,98.0,108.0,104.0,93.0,98.0,100.0,NaN,NaN,NaN,110,99,110,NaN,NaN,NaN
208,18525,2024,Dylan Cease,SDP,28,14,11,4.8,3.47,33,33,1,1,0,0,189.1,762,137,80,73,18,65,0,2,12,0,224,185,197,83,16,1189,1999,3188,95,16,4,1,10.65,3.09,...,21.0,65.0,32.0,NaN,NaN,NaN,60,67,53,-0.48,96.0,102.0,97.0,NaN,NaN,NaN,100.0,102.0,105.0,NaN,NaN,NaN,121.0,105.0,123.0,98.0,107.0,102.0,108.0,81.0,92.0,NaN,NaN,NaN,110,104,113,NaN,NaN,NaN


In [7]:
pitcher_2024

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,...,botOvr FC,botStf FC,botCmd FC,botOvr FS,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
65,10603,2024,Chris Sale,ATL,35,18,3,6.4,2.38,29,29,0,0,0,0,177.2,702,141,48,47,9,39,0,8,4,1,225,191,143,92,8,925,1893,2818,126,5,4,0,11.40,1.98,...,NaN,NaN,NaN,NaN,NaN,NaN,60,48,64,-0.49,100.0,111.0,110.0,NaN,NaN,NaN,97.0,108.0,100.0,95.0,109.0,108.0,114.0,114.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,111,110,NaN,NaN,NaN
66,22267,2024,Tarik Skubal,DET,27,18,4,5.9,2.39,31,31,0,0,0,0,192.0,753,142,54,51,15,35,0,9,2,0,228,218,171,88,22,883,1985,2868,134,13,4,2,10.69,1.64,...,NaN,NaN,NaN,NaN,NaN,NaN,61,59,61,-0.55,127.0,98.0,124.0,NaN,NaN,NaN,104.0,104.0,107.0,111.0,100.0,108.0,108.0,96.0,111.0,98.0,102.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,112,100,112,NaN,NaN,NaN
78,10310,2024,Zack Wheeler,PHI,34,16,7,5.4,2.57,32,32,0,0,0,0,200.0,787,139,62,57,20,52,0,8,8,0,224,213,195,92,27,1079,2057,3136,118,17,3,0,10.08,2.34,...,45.0,61.0,47.0,52.0,46.0,58.0,64,59,60,-0.74,NaN,NaN,NaN,126.0,88.0,106.0,109.0,110.0,118.0,117.0,103.0,122.0,108.0,109.0,117.0,NaN,NaN,NaN,97.0,95.0,96.0,94.0,101.0,91.0,110,104,113,NaN,NaN,NaN
149,21846,2024,Cole Ragans,KCR,26,11,9,4.9,3.14,32,32,1,0,0,0,186.1,762,146,71,65,15,67,1,6,11,0,223,187,186,86,18,1138,1980,3118,86,10,7,2,10.77,3.24,...,51.0,55.0,56.0,NaN,NaN,NaN,58,62,54,-0.38,115.0,92.0,110.0,NaN,NaN,NaN,110.0,101.0,111.0,NaN,NaN,NaN,124.0,102.0,117.0,98.0,108.0,104.0,93.0,98.0,100.0,NaN,NaN,NaN,110,99,110,NaN,NaN,NaN
208,18525,2024,Dylan Cease,SDP,28,14,11,4.8,3.47,33,33,1,1,0,0,189.1,762,137,80,73,18,65,0,2,12,0,224,185,197,83,16,1189,1999,3188,95,16,4,1,10.65,3.09,...,21.0,65.0,32.0,NaN,NaN,NaN,60,67,53,-0.48,96.0,102.0,97.0,NaN,NaN,NaN,100.0,102.0,105.0,NaN,NaN,NaN,121.0,105.0,123.0,98.0,107.0,102.0,108.0,81.0,92.0,NaN,NaN,NaN,110,104,113,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,18403,2024,Enyel De Los Santos,- - -,28,1,2,-0.9,5.20,64,0,0,0,2,2,64.0,278,65,38,37,17,25,1,5,2,0,66,62,87,31,13,381,753,1134,23,5,2,2,9.28,3.52,...,NaN,NaN,NaN,NaN,NaN,NaN,48,41,59,0.24,79.0,87.0,67.0,NaN,NaN,NaN,83.0,108.0,87.0,NaN,NaN,NaN,110.0,102.0,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,104,96,NaN,NaN,NaN
633,20546,2024,Grant Anderson,TEX,27,0,1,-0.9,8.10,23,0,0,0,1,1,26.2,120,33,24,24,11,10,3,1,0,0,29,32,37,11,4,174,309,483,6,2,0,0,9.79,3.38,...,NaN,NaN,NaN,NaN,NaN,NaN,58,54,56,-0.36,83.0,93.0,80.0,NaN,NaN,NaN,96.0,102.0,95.0,122.0,108.0,125.0,106.0,110.0,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,106,109,NaN,NaN,NaN
491,18000,2024,Triston McKenzie,CLE,26,3,5,-1.0,5.11,16,16,0,0,0,0,75.2,340,69,46,43,19,49,1,1,5,0,74,75,111,28,10,534,774,1308,43,6,1,0,8.80,5.83,...,NaN,NaN,NaN,NaN,NaN,NaN,27,37,32,1.54,NaN,NaN,NaN,95.0,91.0,76.0,84.0,83.0,75.0,NaN,NaN,NaN,92.0,71.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,83,75,NaN,NaN,NaN
614,11836,2024,Taijuan Walker,PHI,31,3,7,-1.1,7.10,19,15,0,0,0,0,83.2,381,107,68,66,24,37,0,3,0,0,58,107,115,61,17,543,892,1435,53,9,0,0,6.24,3.98,...,41.0,24.0,44.0,50.0,38.0,52.0,40,31,43,0.76,NaN,NaN,NaN,85.0,110.0,97.0,84.0,82.0,66.0,82.0,97.0,84.0,100.0,96.0,99.0,NaN,NaN,NaN,85.0,90.0,82.0,81.0,105.0,91.0,86,98,87,NaN,NaN,NaN


In [10]:
from pybaseball import playerid_reverse_lookup
import pandas as pd

pitcher_names = pitcher_2024["IDfg"].unique()

pitcher_ids_df = playerid_reverse_lookup(pitcher_names, key_type="fangraphs")

pitcher_ids_df

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,díaz,edwin,621242,diaze006,diazed04,14710,2016.0,2025.0
1,junis,jakob,596001,junij001,junisja01,13619,2017.0,2025.0
2,neris,héctor,593576,nerih001,nerishe01,11804,2014.0,2025.0
3,chavez,jesse,445926,chavj001,chaveje01,5448,2008.0,2025.0
4,assad,javier,665871,assaj001,assadja01,21741,2022.0,2024.0
...,...,...,...,...,...,...,...,...
571,keller,mitch,656605,kellm003,kellemi03,17594,2019.0,2025.0
572,jax,griffin,643377,jax-g001,jaxgr01,20253,2021.0,2025.0
573,kopech,michael,656629,kopem001,kopecmi01,17282,2018.0,2024.0
574,marte,yunior,628708,marty001,marteyu01,14416,2022.0,2024.0


In [6]:
pitcher_ids_df

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,sale,chris,519242,salec001,salech01,10603,2010.0,2025.0
0,skubal,tarik,669373,skubt001,skubata01,22267,2020.0,2025.0
0,wheeler,zack,554430,wheez001,wheelza01,10310,2013.0,2025.0
0,ragans,cole,666142,ragac001,raganco01,21846,2022.0,2025.0
0,cease,dylan,656302,ceasd001,ceasedy01,18525,2019.0,2025.0
...,...,...,...,...,...,...,...,...
0,montero,rafael,606160,montr004,montera01,12760,2014.0,2025.0
0,de los santos,enyel,660853,deloe001,delosen01,18403,2018.0,2025.0
0,anderson,grant,681982,andeg002,andergr01,20546,2023.0,2025.0
0,mckenzie,triston,663474,mcket001,mckentr01,18000,2020.0,2025.0


In [8]:
pitcher_ids_df.to_csv("../data/pitcher_lookup_table.csv", index=False)

In [22]:
import pandas as pd

all_batter_data = pd.read_csv("../data/all_batter_data.csv")

all_batter_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,player_AVG,player_OBP,player_SLG,player_PA
0,FF,2024-09-29,95.8,-0.88,6.01,Jarren Duran,680776,686752,single,hit_into_play,...,NaN,NaN,0.88,0.83,-0.83,46.4,0.285,0.342,0.492,735
1,SI,2024-09-29,92.8,2.36,5.13,Jarren Duran,680776,663992,field_out,hit_into_play,...,NaN,NaN,2.26,1.68,1.68,16.5,0.285,0.342,0.492,735
2,SI,2024-09-29,93.3,2.42,5.22,Jarren Duran,680776,663992,NaN,ball,...,NaN,NaN,2.07,1.62,1.62,16.9,0.285,0.342,0.492,735
3,SI,2024-09-29,92.7,2.41,5.10,Jarren Duran,680776,663992,NaN,foul,...,NaN,NaN,2.62,1.64,1.64,16.1,0.285,0.342,0.492,735
4,SL,2024-09-29,86.3,2.29,4.92,Jarren Duran,680776,663992,NaN,ball,...,NaN,NaN,2.85,-0.03,-0.03,11.1,0.285,0.342,0.492,735


In [23]:
pitcher_ids = all_batter_data["pitcher"].unique()

len(pitcher_ids)

384

In [24]:
from pybaseball import playerid_reverse_lookup

pitcher_ids_df_full = playerid_reverse_lookup(pitcher_ids, key_type="mlbam")

pitcher_ids_df_full

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,díaz,edwin,621242,diaze006,diazed04,14710,2016.0,2025.0
1,chavez,jesse,445926,chavj001,chaveje01,5448,2008.0,2025.0
2,turnbull,spencer,605513,turns003,turnbsp01,16207,2018.0,2024.0
3,tyler,kyle,667725,tylek001,tylerky01,22092,2021.0,2024.0
4,wisdom,patrick,621550,wisdp001,wisdopa01,13602,2018.0,2024.0
...,...,...,...,...,...,...,...,...
379,keller,mitch,656605,kellm003,kellemi03,17594,2019.0,2025.0
380,jax,griffin,643377,jax-g001,jaxgr01,20253,2021.0,2025.0
381,kopech,michael,656629,kopem001,kopecmi01,17282,2018.0,2024.0
382,eisert,brandon,685126,eiseb001,eiserbr01,-1,2024.0,2025.0


In [13]:
missing_ids = set(pitcher_ids) - set(pitcher_ids_df_full["key_mlbam"].unique())
print("Missing IDs:", missing_ids)

Missing IDs: set()


In [9]:
pitcher_ids_df_full.to_csv("../data/pitcher_lookup_table.csv", index=False)

In [26]:
pitcher_stats = pd.read_csv("../data/all_pitcher_data.csv")

In [27]:
missing_ids = set(pitcher_ids_df_full["key_mlbam"].unique()) - set(pitcher_stats["mlbID"])
print("Missing IDs:", missing_ids)

Missing IDs: set()
